# Key-Value Attention Mechanism Homework on Keras: Character-level Machine Translation (Many-to-Many, encoder-decoder)

In this homework, you will create an MT model with key-value attention mechnism that coverts names of constituency MP candidates in the 2019 Thai general election from Thai script to Roman(Latin) script. E.g. นิยม-->niyom 

In [42]:
# !wget https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

import matplotlib as mpl
import matplotlib.font_manager as fm

fm.fontManager.addfont('thsarabunnew-webfont.ttf') # 3.2+
mpl.rc('font', family='TH Sarabun New')

2.11.0


In [43]:
%matplotlib inline
import keras
import numpy as np
import random
np.random.seed(0)

from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical, pad_sequences
from keras.models import load_model, Model
from keras import backend as K

# %matplotlib inline
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
# from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras import Model
# from tensorflow.keras.models import load_model
# import tensorflow.keras.backend as K
# import numpy as np

# import random

## Load Dataset
We have generated a toy dataset using names of constituency MP candidates in 2019 Thai General Election from elect.in.th's github(https://github.com/codeforthailand/dataset-election-62-candidates) and tltk (https://pypi.org/project/tltk/) library to convert them into Roman script.

<img src="https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW8/images/dataset_diagram.png" alt="Drawing" style="width: 500px;"/>


In [44]:
# from google.colab import drive
# drive.mount('/content/drive')

# import shutil
# shutil.copy("/content/drive/MyDrive/FRA 501 IntroNLP&DL/Dataset/mp_name_th_en.csv", "/content/mp_name_th_en.csv")

In [45]:
import csv
with open('data/mp_name_th_en.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    name_th = []
    name_en = []
    for row in readCSV:
        name_th.append(row[0])
        name_en.append(row[1])

In [46]:
for th, en in zip(name_th[:10],name_en[:10]):
    print(th,en)

ไกรสีห์ kraisi
พัชรี phatri
ธีระ thira
วุฒิกร wutthikon
ไสว sawai
สัมภาษณ์  samphat
วศิน wasin
ทินวัฒน์ thinwat
ศักดินัย sakdinai
สุรศักดิ์ surasak


## Task1: Preprocess dataset for Keras
* 2 dictionaries for indexing (1 for input and another for output)
* DON'T FORGET TO INCLUDE special token for padding
* DON'T FORGET TO INCLUDE special token for the end of word symbol (output)
* Be mindful of your pad_sequences "padding" hyperparameter. Choose wisely (post-padding vs pre-padding)

In [47]:
#FILL YOUR CODE HERE

input_chars = list(set(''.join(name_th)))
output_chars = list(set(''.join(name_en)))

data_size, vocab_size = len(name_th), len(input_chars)+1 # 1 for padding
output_size, output_vocab_size = len(name_en), len(output_chars)+ 2 # 1 for padding 1 for end of word

print('data has %d names, %d unique characters in input, %d unique characters in output.' % (data_size, vocab_size, output_vocab_size))
maxlen = len( max(name_th, key=len)) #max input length
print('max input length is', maxlen)

data has 10887 names, 65 unique characters in input, 24 unique characters in output.
max input length is 20


In [48]:
sorted_input_chars = sorted(input_chars)
sorted_output_chars = sorted(output_chars)

sorted_input_chars.insert(0,"<PAD>") #PADDING
sorted_output_chars.insert(0,"<EOS>") #END OF WORD
sorted_output_chars.insert(0,"<PAD>") #PADDING

# Input dictionary
input_char2idx = dict((c, i) for i, c in enumerate(sorted_input_chars))
input_idx2char = dict((i, c) for i, c in enumerate(sorted_input_chars))
# Output dictionary
output_char2idx = dict((c, i) for i, c in enumerate(sorted_output_chars))
output_idx2char = dict((i, c) for i, c in enumerate(sorted_output_chars))

print('input_char_indices', input_idx2char)
print('output_char_indices', output_idx2char)

input_char_indices {0: '<PAD>', 1: ' ', 2: 'ก', 3: 'ข', 4: 'ค', 5: 'ฆ', 6: 'ง', 7: 'จ', 8: 'ฉ', 9: 'ช', 10: 'ซ', 11: 'ฌ', 12: 'ญ', 13: 'ฎ', 14: 'ฏ', 15: 'ฐ', 16: 'ฑ', 17: 'ฒ', 18: 'ณ', 19: 'ด', 20: 'ต', 21: 'ถ', 22: 'ท', 23: 'ธ', 24: 'น', 25: 'บ', 26: 'ป', 27: 'ผ', 28: 'ฝ', 29: 'พ', 30: 'ฟ', 31: 'ภ', 32: 'ม', 33: 'ย', 34: 'ร', 35: 'ล', 36: 'ว', 37: 'ศ', 38: 'ษ', 39: 'ส', 40: 'ห', 41: 'ฬ', 42: 'อ', 43: 'ฮ', 44: 'ะ', 45: 'ั', 46: 'า', 47: 'ำ', 48: 'ิ', 49: 'ี', 50: 'ึ', 51: 'ื', 52: 'ุ', 53: 'ู', 54: 'เ', 55: 'แ', 56: 'โ', 57: 'ใ', 58: 'ไ', 59: '็', 60: '่', 61: '้', 62: '๊', 63: '๋', 64: '์'}
output_char_indices {0: '<PAD>', 1: '<EOS>', 2: '-', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'k', 13: 'l', 14: 'm', 15: 'n', 16: 'o', 17: 'p', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'w', 23: 'y'}


In [49]:
print('samplesize', data_size)
Tx = maxlen
# Ty = len(max(name_en, key=len)) # max output length
Ty = Tx * 2
print('max input length is', Tx)
print('max output length is', Ty)

samplesize 10887
max input length is 20
max output length is 40


In [50]:
# Padding
X = []
for name in name_th:
    temp = []
    for char in name:
        temp.append(input_char2idx[char])
    X.append(temp)

Y = []
for name in name_en:
    temp = []
    for char in name:
        temp.append(output_char2idx[char])
    Y.append(temp)

print(f'Befor padding:')
print('X', X[:3])
print('Y', Y[:3])

# We choose padding='post' because we want to pad after the sentence
X = pad_sequences(X, maxlen=Tx, padding='post', value=0)
Y = pad_sequences(Y, maxlen=Ty, padding='post', value=0)

print(f'After padding:')
print('X', X[:3])
print('Y', Y[:3])

Befor padding:
X [[58, 2, 34, 39, 49, 40, 64], [29, 45, 9, 34, 49], [23, 49, 34, 44]]
Y [[12, 18, 3, 11, 19, 11], [17, 10, 3, 20, 18, 11], [20, 10, 11, 18, 3]]
After padding:
X [[58  2 34 39 49 40 64  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [29 45  9 34 49  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [23 49 34 44  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
Y [[12 18  3 11 19 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [17 10  3 20 18 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [20 10 11 18  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


In [51]:
# One-hot encoding
X = to_categorical(X, num_classes=vocab_size)
Y = to_categorical(Y, num_classes=output_vocab_size)

X = X.reshape(data_size, Tx, vocab_size)
Y = Y.reshape(data_size, Ty, output_vocab_size)

print(f'X shape: {X.shape}')
print(f'Y shape: {Y.shape}')

X shape: (10887, 20, 65)
Y shape: (10887, 40, 24)


# Attention Mechanism
## Task 2: Code your own (key-value) attention mechnism
* PLEASE READ: you DO NOT have to follow all the details in (Daniluk, et al. 2017). You just need to create a key-value attention mechanism where the "key" part of the mechanism is used for attention score calculation, and the "value" part of the mechanism is used to encode information to create a context vector.  
* Define global variables
* fill code for one_step_attention function
* Hint: use keras.layers.Lambda 
* Hint: you will probably need more hidden dimmensions than what you've seen in the demo


In [ ]:
from keras.activations import softmax
from keras.layers import Lambda

def softMaxAxis1(x):
    return softmax(x,axis=1)

In [ ]:
#These are global variables (shared layers)
## Fill your code here
## you are allowed to use code in the demo as your template. 

#repeater = ???
#concatenator = ???
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)

#Key-values (Hint)
splitter = Lambda(lambda x:tf.split(x, num_or_size_splits=2,aixs=2)) 

#fatten_1 = ???
#fatten_2 = ???
fatten_1 = Dense(1, activation = "tanh")
fatten_2 = Dense(1, activation = "relu")

#activator = ???
activator = Activation(softMaxAxis1, name='attention_score') # We are using a custom softmax(axis = 1) loaded in this notebook
#dotor = ???
dotor = Dot(axes = 1)



In [ ]:
def one_step_attention(a, s_prev):

    #Fill code here
    #key, value = ???


    #concat = ...key...
    #context = ...value...

    # Repeat the decoder hidden state to concat with encoder hidden states
    s_prev = repeator(s_prev)
    concat = concatenator([a,s_prev])
    # attention function
    e = fattn_1(concat)
    energies =fattn_2(e)
    # calculate attention_scores (softmax)
    attention_scores = activator(energies)
    #calculate a context vector
    context = dotor([attention_scores,a])

    return None # return whatever you need to complete this homework 

## Task3: Create and train your encoder/decoder model here
* HINT: you will probably need more hidden dimmensions than what you've seen in the demo

In [ ]:
#FILL CODE HERE :Hint --> heatmap in CNN + GradCAM

# def model(Tx, Ty, n_h, n_s, vocab_size, machine_vocab_size):
#   ...

# def inference_encoder(Tx, Ty, n_h, n_s, vocab_size, machine_vocab_size):
#    X = ....
#    h = ....

#    model = Model(inputs=[X],outputs=h)
#    return model

# def inference_decoder(Tx, Ty, n_h, n_s, vocab_size, machine_vocab_size):
#   s0 = ...
#   c0 = ...
#   h = ...
#   context, attention_scores, energies = one_step_attention(h, s)
#   ...decoder_LSTM_cell...
#   out = output_layer(s)

#   model = Model(inputs=[h,s0,c0],outputs=[out,s,c,atten_score,energies])

#   return model


In [ ]:
#FIT YOUR MODEL HERE

# Thai-Script to Roman-Script Translation
* Task 4: Test your model on 5 examples of your choice including your name! 
* Task 5: Show your visualization of attention scores on one of your example 

In [ ]:
#task 4
#fill your code here


### Plot the attention map
* If you need to install thai font: sudo apt install xfonts-thai
* this is what your visualization might look like:
--> https://drive.google.com/file/d/168J5SPSf4NNKj718wWUEDpUbh8QYZKux/view?usp=share_link

In [ ]:
# EXAMPLES = ???
# h = inferEncoder_model.predict(EXAMPLES)
# s0 = ???
# c0 = ???
# ...
# Ty = 10
# for t in range(Ty):
#   out,s,c,attention_scores,energies = inferDecoder_model.predict([h,s0,c0])
# ...


In [ ]:
#task 5
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family']='TH Sarabun New'  #you can change to other font that works for you
#fill your code here